# Introduction to Biopython
<center> <img src="img/logo_biopython.PNG" width="500" height="300"/> </center>


Biopython is a set of freely available Python tools that help us to work with biological data. It contains components that are developed specifically for bioinformatics purposes. Today I'll give you the highlights and the rest will be yours to explore. 

On a small side note, before diving into your (biological) data and try analyzing it with complex self-written scripts, it often makes sense that you search through the official documentation for the parts that might help you. This is because it's a huge library that can do a variety of things such as sequence analysis, multiple alignments, protein structures, phylogenetics, population genetics, etc. etc. 

**Installing** the complete module can be done:
- In the Environments section of Anaconda Navigator and manually selecting the package, 
- Installing in the Conda environment: `conda install -c conda-forge biopython`, 
- Installing in the Jupyter Notebook: `pip install biopython`

In [ ]:
pip install biopython 

In [ ]:
# Import the Biopython library
import Bio

# Check version for proper installment (v1.74)
print(Bio.__version__)

it makes sense to install functions or submodules that are part of Biopython in order to ease the use. Imagine that you want to work with sequences, you can import only the Seq-object.

**Content for today**
- Working with sequences in : `Seq` and `Alphabets`
- Sequence annotations with: `SeqRecord` objects
- Reading, writing and parsing files with: `SeqIO`
- Querying NCBI with: `SeqIO`
- BLAST from within Python

## Working with sequences
- A sequence is stored in a `Seq`-object

Biological sequences (DNA, RNA, proteins) are arguably the most important object in bioinformatics. Hence biggest part of Biopython is built around sequences as wel. So in Biopython a sequence is stored in a so-called Sequence object. 
Minor note: Before Oct 2020, each `Seq` object was associated with a specific alphabet type to identify its biological origin (DNA, RNA, Protein, and all derivatives), however `Bio.Alphabet` has now been removed from Biopython. 

Change `type(my_seq)` to `dir(my_seq)`.

In [7]:
# Imports
from Bio.Seq import Seq

In [8]:
# Creating our first Seq object
my_seq = Seq("AGTACACTGGAT")
my_seq

Seq('AGTACACTGGAT')

In [12]:
type(my_seq)

Bio.Seq.Seq

## Working with sequences
- `Seq`-objects behave like strings

Once created, `Seq` objects can be used as if they were normal Python strings regarding retrieving the length of the sequence or iterating over the characters

In [14]:
# Get element at position 0
my_seq = Seq('AGTACACTGGAT')
my_seq[0]

# Find how many times "GAT" appears in the sequence
print(my_seq.count("GAT"))

# Find where "GAT" appears in the sequence
print(my_seq.find("GAT"))

1
9


- `Seq`-objects are immutable

In [ ]:
my_seq[2] = 'A'

## Working with sequences
- `Seq`-objects can be sliced

Notes: slicing following conventions of Python : starting at 0 with first item included en last item excluded. Also do: my_seq[0::3], my_seq[::-1], ...

In [19]:
# Slicing in its most basic form
my_seq[2:6]

Seq('TACA')

- Turning `Seq`-objects into strings

In [20]:
str(my_seq)
#print(my_seq)

'AGTACACTGGAT'

- Concatenating or adding sequences

In [ ]:
# This will work
protein_seq1 = Seq("EVRNAK")
protein_seq2 = Seq("AGGATC")
protein_seq1 + protein_seq2

- and much more...

## Methods on `Seq` objects
- transcribe()
- translate()
- complement()
- reverse_complement()



![transcription](img/transcriptionprocess.png)

In [24]:
coding_dna = Seq("ATGATCTCGTAA")
#dir(coding_dna)
print(f'Original DNA seq: {str(coding_dna):>26}')
print(f"Complement DNA seq: {str(coding_dna.complement()):>24}")
print(f"Reverse complement DNA seq: {str(coding_dna.reverse_complement()):>16}")
print(f"mRNA seq: {str(coding_dna.transcribe()):>34}")
print(f"Protein seq: {str(coding_dna.translate()):>23}")

Original DNA seq:               ATGATCTCGTAA
Complement DNA seq:             TACTAGAGCATT
Reverse complement DNA seq:     TTACGAGATCAT
mRNA seq:                       AUGAUCUCGUAA
Protein seq:                    MIS*


Notes: as described in the notebook materials, it is technically possible to do a reverse complement or even a back-transcription process on a protein sequence. That really doesn't make sense, although Biopython will not object this. 

## Translation and codon tables
- Choose the correct codon table that is relevant for the organism you're working with.   
- Imported from NCBI: standard, vertebrate mictochondrial, yeast mitochondrial, bacterial, etc. 

In [29]:
# Example
mRNA = Seq('AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUUGA')
print(mRNA.translate(table="Standard")) # This is the default and does not need to be specified
print(mRNA.translate(table="Vertebrate Mitochondrial")) 

MAIVMGR*KG*
MAIVMGRWKGW


Note: Suppose we are dealing with a mitochondrial sequence. We need to tell the translation function to use the relevant genetic code instead. Let's first inspect the vertebrate mitochondrial codon table and then translate the same coding DNA sequence. 

In [25]:
# Import codon table ()
from Bio.Data import CodonTable
# Change 'Standard' codon table to 'Vertebrate Mitochondrial'
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
print(standard_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

 Change to "Vertebrate Mitochondrial".

In [35]:
# specify the table using the NCBI ID or table number (e.g. 2)
mRNA = Seq('AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUUGA')
mRNA.translate(table=1, to_stop=True)

Seq('MAIVMGR')

Note: use  `table=1`, then `to_stop=True`, you can even change the interpreted `stop_symbol="@"`. 

Example: gene yaaX in E. coli K12.

In [42]:
gene = Seq("GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA" + \
 "GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT" + \
 "AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT" + \
 "TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT" + \
 "AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA")
gene.translate()

Seq('VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDH...HR*')

Notes: even though we're changing the table to bacterial (which is appropriate) still we're not taking into account the behaviour of this E. coli where GTG is a valid start codon (Valine --> Methionine)
```python
gene.translate(table="Bacterial", to_stop=True, cds=True)
```
Another possible option is the argument `cds` that will claim that this is a valid CDS (start, stop codon, expected number of tri-nts for the codons with no internal stop-codons.

# Exercises
- 5.2.1 +
- 5.3.1 +
- 5.7 +++ 

# 6. Sequence annotations
- `Seq` = sequence 
- `SeqRecord` = `Seq` + metadata
- `SeqRecord`
    - Main attributes: id & seq
    - Additional attributes: name, description, dbxrefs, features, annotations

In [ ]:
# Imports
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

Uptil now, we've been using sequence (Seq) objects that stored a sequence and the file format (i.e. fasta, genbank, etc.). Biopython allows us to annotate these Seq objects with additional information like an identifier, a name of the sequence, a description, features and ultimately a bunch of annotations. All of this information is stored in the so-called SeqRecord object which is the follow-up of the Seq object.

In [ ]:
# Import SeqRecord object with the SeqIO module
from Bio import SeqIO
record = SeqIO.read("data/NC_005816.gb","gb")
print(record)

In this case we'll read in a GenBank file, *NC_005816.gb*, which we’ll load using the SeqIO module. It's accessible in [NCBI](https://www.ncbi.nlm.nih.gov/nuccore/NC_005816). The next chapter will discuss the SeqIO module, however here we're just using it to read in a SeqRecord object from a file. 

The following elements are present (amongst others):
- **ID**: usually the accession number of the sequence
- **Name**: the more commonly used name of the sequence (often the same as accession number)
- **Description**: a description or expressive name for the sequence
- **Features**: a list of SeqFeature objects with more structured information about the sequence (discussed below)
- **Annotations**: a dictionary of additional information about the sequence. 
- **Seq**: the sequence itself

In [ ]:
# ID
print(record.id)
# Name
print(record.name)
# Description
print(record.description)
# Features
#print(record.features)
# Annotations
record.annotations['taxonomy']
# Sequence
record.seq

The features and their SeqFeature object are a fairly complex thing on their own. Basically they contain more abstract and detailed information about the SeqRecord object (and thus the sequence). It attempts to encapsulate as much of the information about the sequence as possible by describing a region on the parent sequence.

The features and their SeqFeature object are a fairly complex thing on their own. Basically they contain more abstract and detailed information about the SeqRecord object (and thus the sequence). It attempts to encapsulate as much of the information about the sequence as possible by describing a region on the parent sequence.
It allows us e.g. to extract CDSs from a longer sequence. 

Example SeqFeatures:

In [ ]:
# Extract features and check whether SNP of interest (4350) is present
my_snp = 4350
record = SeqIO.read("data/NC_005816.gb", "genbank")

for feature in record.features:
    if my_snp in feature: 
        print(f"Feature type: {feature.type}, Locus tag(s):  {feature.qualifiers.get('locus_tag')}")

**db_xref** - A list of database cross-references as strings.

**Locus tags** are identifiers applied systematically to every gene in a sequencing project. If two submitters of different genomes use the same systematic names to describe different genes, this can be a source of confusion. Therefore, INSDC maintains a registry of locus tag prefixes to avoid overlap between genome annotation projects. The prefix is then used systematically to give a new unambiguous name to every gene.

# Exercises
- 6.1.1 ++

## Reading, writing and parsing files
- `SeqIO` reads, writes and parses `SeqRecord` objects
- The output of `Bio.SeqIO.parse()` is a `SeqRecord` iterator.
- The output of `Bio.SeqIO.read()` is one `SeqRecord`object

In [ ]:
# Import
from Bio import SeqIO

If there is only one record in the file, you might as well use the Bio.SeqIO.read() function. It takes the same two arguments and returns a SeqRecord object with one record.

Example parsing:

In [ ]:
# Here we're using the explicit path to a fasta file (in the data folder)
for seq_record in SeqIO.parse("data/ls_orchid.fasta", "fasta"):
    print(seq_record.id)

Alternatively:

In [ ]:
with open("data/ls_orchid.fasta", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        print(record.id)

Alternatively

In [ ]:
# List comprehension
identifiers = [seq_record.id for seq_record in SeqIO.parse("data/ls_orchid.gbk","genbank")]

# Exercises
- 7.1.1 +

The first thing we'll want to do is reading in a sequence object. The function that we'll need for that is `Bio.SeqIO.parse()` and it expects two arguments:
1. An explicit path to a file, a filehandle or a link to data that can be downloaded from the internet  
2. A lower case string specifying the sequence format. Examples are: clustal, fasta, embl, fastq, genbank or gb, pdb-atom, swiss, uniprot-xml,... You must specify the file format because [*explicit is better than implicit*](https://www.python.org/dev/peps/pep-0020/). 

## Parsing from the internet
- Download and parse sequences from internet (NCBI, Swiss-prot, ExPASy, etc.)
- Big files: fetch once and store
- [Entrez](https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html):
    - data retrieval system 
    - provides users access to NCBI's databases: PubMed, GenBank, GEO, and many others.
- Output typically in `XML`

In [ ]:
# Imports
from Bio import Entrez
from Bio import SeqIO

In the previous sections, we looked at parsing sequence data from a file (using a filename or handle). As discussed in the introduction of this chapter, it's also possible to download and parse sequences from the internet. Note that just because you can download sequence data and parse it into a SeqRecord object in one go doesn't mean this is a good idea. In general, you should probably download sequences once and save them to a file for reuse.

You can access Entrez from a web browser to manually enter queries, or you can use Biopython's Bio.Entrez module for programmatic access to Entrez. 

- `Entrez.email`
- `Entrez.esearch`
- `Entrez.efetch` 
- `Entrez.read`
- `Entrez.parse`
- ...

In [ ]:
# Provide email address
Entrez.email = "hello@its.me"

# Use e-search to search any of the databases of NCBI
with Entrez.esearch(db="nucleotide", term="Cypripedioideae[Orgn] AND matK[Gene]", idtype="acc") as handle:
    records = Entrez.read(handle)

In [ ]:
records

Output is a record which can be parsed

In [ ]:
Entrez.email = "hello@its.me"

with Entrez.efetch(db="nucleotide", rettype="fasta", retmode="text", id="6273291") as handle:
    record = SeqIO.read(handle, "fasta")
record

In [ ]:
Entrez.email = "hello@its.me"

with Entrez.efetch(db="nucleotide", id="6273291", retmode="xml", rettype="fasta") as handle:
    record = Entrez.read(handle)
record

First, we'll have to define who we are. Then, we're telling the e-fetching function that we want to access the nucleotide database, we're looking for a fasta sequence, the file format and the id in the form of a handle, which we will subsequently read with the Bio.SeqIO.read() function.

# Exercises
- Sickle cell ++ (long)

## BLAST in Biopython
- e.g. find out which organism a sequence belongs to
- Function `qblast()` part of `Bio.Blast.NCBIWWW`

Last thing we will see for today is BLAST. And BLAST represents Basic Local Alignment Search Tool. So what it does is, imagine that you obtained a sequence in the lab, BLAST will find regions of similarity between your sequence and a database full of sequences by trying to  align your sequence to the one of the database. Together with this alignment it will also calculate a statistical significance of how likely that the two sequences are identical, because you can expect that there will be mutliple possible alignments, one a bit more likely than the other (based on mismatches during the alignment procedures).

The function that we will use for this is qblast and it's part of BIo Blast NCBI-WWW. 

In [ ]:
# Imports
from Bio.Blast import NCBIWWW

- Three arguments: 
    1. blast program: blastn, blastp, blastx, etc
    2. databases: e.g. nt for nucleotide
    3. sequence, sequence in fasta or identifier

In [ ]:
# Don't run this all at once
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")
#result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
#result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

- Takes a long time to run

- Output in handle object (by default XML)
- Next step:
    - read/parse XML output into Python objects
    - Save a local copy
- handle object output can only be used once - calling it again returns an empty string.

This is especially useful when debugging code that extracts info from the BLAST results (because re-running the online search is slow and wastes the NCBI computer time).

Parse XML output into Python objects:

In [ ]:
from Bio.Blast import NCBIXML

# Option 1:
blast_records = NCBIXML.parse(result_handle)

# Alternatively, for only one BLAST result:
blast_record = NCBIXML.read(result_handle)

Save a local copy:

In [ ]:
# Option 2:
with open("file.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
    
with open("file.xml", "r") as fh:
    my_blast = fh.read()

The Blast record holds the information of the BLAST output, e.g.:
- Description: information about one hit description,
- Alignment: information about one alignment hit, 
- HSP: information about one HSP. 


In [ ]:
from Bio.Blast import NCBIXML
# This is a blast result output that has been saved in a file, called my_blast.xml
with open("data/my_blast.xml", "r") as fh:
    # Read in records
    blast_records = NCBIXML.parse(fh)
    for blast_record in blast_records:
        # Extract information!
        for descr in blast_record.descriptions:
            title = descr.title
            score = descr.score
            e = descr.e
            print(title, score, e)